<a href="https://colab.research.google.com/github/felipeescorel/redes_neurais/blob/master/MLP/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [0]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

In [2]:
!git clone https://github.com/felipeescorel/redes_neurais.git

Cloning into 'redes_neurais'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 146 (delta 9), reused 20 (delta 1), pack-reused 104
Receiving objects: 100% (146/146), 34.11 MiB | 22.26 MiB/s, done.
Resolving deltas: 100% (29/29), done.


A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


# **Rede 1**

In [0]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a,tipo_fun):
        """Retorna a saída da rede se `a` for entrada."""
        for b, w in zip(self.biases, self.weights):
            a = funcao_ativacao(tipo_fun,False,np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None,tipo_fun='sigmoid'):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
        starting_eta = eta
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta,tipo_fun)
            eta = learning_upd(starting_eta,j,epochs)
            if test_data:
              print("Epoch {} : {} / {}".format(j,self.evaluate(test_data,tipo_fun),n_test));
            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, eta,tipo_fun):
        """Atualiza os pesos e bias da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `eta` é a taxa de aprendizado."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y,tipo_fun)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y,tipo_fun):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo C_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores z, camada por camada
        zs = [] 

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = funcao_ativacao(tipo_fun,False,z)
            activations.append(activation)
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * funcao_ativacao(tipo_fun,True,zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante. 
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = funcao_ativacao(tipo_fun,True,z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data,tipo_fun):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x,tipo_fun)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

# # Função de Ativação Sigmóide
# def sigmoid(z):
#     return 1.0/(1.0+np.exp(-z))

# # Função para retornar as derivadas da função Sigmóide
# def sigmoid_prime(z):
#     return sigmoid(z)*(1-sigmoid(z))


In [0]:
def funcao_ativacao(tipo, is_prime, z):
  # Função de Ativação Sigmóide
  def sigmoid(z):
      return 1.0/(1.0+np.exp(-z))

  def relu(z):
      return np.array([[max(0, i[0])] for i in z])

  # Função para retornar as derivadas da função Sigmóide
  def sigmoid_prime(z):
      return sigmoid(z)*(1-sigmoid(z))

  def relu_prime(z):
      return relu(z)*(1-relu(z))
  
  switcher = {
      'sigmoid': sigmoid,
      'relu': relu
  }
  
  switcher_prime = {
      'sigmoid': sigmoid_prime,
      'relu': relu_prime,
  }
  
  if is_prime:
    fun = switcher_prime.get(tipo)
  else:
    fun = switcher.get(tipo)
  return fun(z)


def learning_upd(taxa_inicial, epoch, n_epochs):
    return taxa_inicial * np.exp(-epoch / n_epochs)

Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [0]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
  
def all_data():
  training_data, validation_data, test_data = load_data_wrapper()

  training_data = list(training_data)
  validation_data = list(validation_data)
  test_data = list(test_data)

  return training_data + validation_data + test_data


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (eta)




In [6]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)


net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)



Epoch 0 : 1019 / 10000
Epoch 1 : 1007 / 10000
Epoch 2 : 988 / 10000
Epoch 3 : 1014 / 10000
Epoch 4 : 1007 / 10000
Epoch 5 : 1008 / 10000
Epoch 6 : 1027 / 10000
Epoch 7 : 1013 / 10000
Epoch 8 : 997 / 10000
Epoch 9 : 1001 / 10000
Epoch 10 : 1023 / 10000
Epoch 11 : 1010 / 10000
Epoch 12 : 1000 / 10000
Epoch 13 : 1021 / 10000
Epoch 14 : 1011 / 10000
Epoch 15 : 998 / 10000
Epoch 16 : 1026 / 10000
Epoch 17 : 1002 / 10000
Epoch 18 : 1025 / 10000
Epoch 19 : 1011 / 10000
Epoch 20 : 1004 / 10000
Epoch 21 : 1003 / 10000
Epoch 22 : 1011 / 10000
Epoch 23 : 1008 / 10000
Epoch 24 : 1008 / 10000
Epoch 25 : 1002 / 10000
Epoch 26 : 1011 / 10000
Epoch 27 : 997 / 10000
Epoch 28 : 1011 / 10000
Epoch 29 : 1000 / 10000


##Mini-Projeto

1) Realizar avaliaçao usando validação cruzada

2) Modificar a taxa de aprendizagem pra diminuir com o tempo

3) Realizar avaliações com a validação cruzada modificando os seguintes parâmetros:
     
     a) Taxa de aprendizagem: 0.5, 0.1 e 0.001
     b) Função de ativação RELU
     c) 3 diferentes configurações de neurônios na camada  intermediária


Data de Entrega: 23/04/2019

A partir dos melhores resultados obtidos na etapa anterior, resolver as seguintes questões:

4) Implementar as regularizações L1 e L2

5) Implementar o Momento

6) Comparar os experimentos e explicar o que achou dos resultados.

Data de Entrega: 30/04/2019


In [7]:
data = all_data()
net = Network([784, 30, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [8]:
data = all_data()
net = Network([784, 30, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.1
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [9]:
data = all_data()
net = Network([784, 30, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.4
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [10]:
data = all_data()
net = Network([784, 40, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [11]:
data = all_data()
net = Network([784, 40, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [12]:
data = all_data()
net = Network([784, 40, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [13]:
data = all_data()
net = Network([784, 60, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in multiply


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [14]:
data = all_data()
net = Network([784, 60, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.1
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
K 3
Epoch 0 : 17500 / 17500


In [15]:
data = all_data()
net = Network([784, 60, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.01
epoch = 1

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1852 / 17500
K 1
Epoch 0 : 1780 / 17500
K 2
Epoch 0 : 1543 / 17500
K 3
Epoch 0 : 0 / 17500


**PARTE 2**

In [0]:
# Classe Network
class Network2(object):

    def __init__(self, sizes, regularizacao='L1', lambda_value=0.00001, beta=0.7):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.lambda_value = lambda_value
        self.regularizacao = regularizacao
        self.beta = beta

    def feedforward(self, a, tipo_fun):
        """Retorna a saída da rede se `a` for entrada."""
        for b, w in zip(self.biases, self.weights):
            a = funcao_ativacao(tipo_fun, False, np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None,
            tipo_fun='sigmoid'):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
        
        eta_inicial = eta
        training_data = list(training_data)
        n = len(training_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, tipo_fun)
                        
            eta = learning_upd(eta_inicial, j, epochs)
        
          
        if test_data:
            predic = self.evaluate(test_data, tipo_fun)
            print("Total Input {} / Total predict {}".format(len(test_data), predic))
            return predic
        else:
            print("Epoch {} finalizada".format(j))
               
    def update_mini_batch(self, mini_batch, eta, tipo_fun):
        """Atualiza os pesos e bias da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `eta` é a taxa de aprendizado."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
          delta_nabla_b, delta_nabla_w = self.backprop(x, y, tipo_fun)
          nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
          nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        pos = 0
#         momento = []
        for w, nw in zip(self.weights, nabla_w):
          constante = self.lambda_value

          if self.regularizacao == 'L2':
            constante *= w 
          
#           if len(momento) == 0:
#             v_t = self.beta
#           else:
#             v_t = self.beta*momento[pos-1] + self.weights[pos-1]
          
#           momento.append(v_t)
#           peso = w-(eta/len(mini_batch))*nw
#           self.weights[pos] = (peso + constante) - v_t
          self.weights[pos] = w-(eta/len(mini_batch))*nw + constante
          pos += 1

        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        

    def backprop(self, x, y, tipo_fun):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo C_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores z, camada por camada
        zs = [] 

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = funcao_ativacao(tipo_fun, False, z)
            activations.append(activation)
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * funcao_ativacao(tipo_fun, True, zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante. 
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = funcao_ativacao(tipo_fun, True, z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data, tipo_fun):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x, tipo_fun)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)

In [20]:
data = all_data()
net = Network2([784, 30, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 10

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: overflow encountered in add
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [21]:
data = all_data()
net = Network2([784, 30, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.1
epoch = 10

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1699
K 1
Total Input 17500 / Total predict 1735
K 2
Total Input 17500 / Total predict 1534
K 3
Total Input 17500 / Total predict 0


In [22]:
data = all_data()
net = Network2([784, 30, 10])
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.001
epoch = 10

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1678
K 1
Total Input 17500 / Total predict 1695
K 2
Total Input 17500 / Total predict 1469
K 3
Total Input 17500 / Total predict 0


In [23]:
data = all_data()
net = Network2([784, 30, 10],regularizacao='L2')
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.5
epoch = 10

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [24]:
data = all_data()
net = Network2([784, 30, 10],regularizacao='L2')
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.1
epoch = 10

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [25]:
data = all_data()
net = Network2([784, 30, 10],regularizacao='L2')
acertos_dict = {}
crossValidation = KFold(n_splits= 4)
at = 0.001
epoch = 10

for i, index in enumerate(crossValidation.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500
